# Taller N-Grams

1. Realizar un programa para computar unigramas, bigramas y trigramas. Los parámetros son: corpus y N (tamaño del "gram") y debe generar el modelo de lenguaje basado en el conteo de los n-grams. La salida puede ser un diccionario con la siguiente estructura:

```
 Si N=2
     { (A,F) : 0.2,
       (D,F) : 0.15,
       (B,G) : 0.3,
       (B,H) : 0.1,
     }
```
En este caso para el bigrama `(A,F)` la `P(F|A) = 0.2` y para el bigrama `(D,F)` la `P(F|D) = 0.15` y así para todos los bigramas. No incluir en el diccionario de salida las `P(X|Y)=0` (podrían ser muchas).

In [1]:
# Initializing
import re
import collections
import operator, functools
from nltk.tokenize import WordPunctTokenizer

# Tuple flattening utility
def flatten(t):
    for element in t:
        if isinstance(element, collections.Iterable) and not isinstance(element,str):
            for x in flatten(element):
                yield re.sub(r"\[\d+\]",r"",x)
        else:
            yield re.sub(r"\[\d+\]",r"",element)
            
def printer(corpus, result):
    print('Corpus:')
    for phrase in corpus:
        print(phrase)
    print('\nTokens count:')
    for token in result['tokens']:
        print(token, ':', result['tokens'][token])

    print('\nGrams')
    for gram in result['grams']:
        print(gram, ':', result['grams'][gram])

In [2]:
# Gram generator
def grams(corpus, gram):
    individual_count = {}
    gram_count = {}
    grams_probs = {}
    for phrase in corpus:
        for token in phrase:
            if token in individual_count:
                individual_count[token] += 1
            else:
                individual_count[token] = 1
        lists = []
        for i in range(0, gram):
            lists.append(phrase[i:])
        tuples = lists.pop(0)
        for item in lists:
            tuples = list(zip(tuples, item))
        if gram == 1:
            tuples = list(map(lambda x: tuple([x]), tuples))
        for item in tuples:
            ngram = tuple(flatten(list(item)))
            if ngram in gram_count:
                gram_count[ngram]['count'] += 1
            else:
                gram_count[ngram] = {}
                gram_count[ngram]['count'] = 1
                gram_count[ngram]['probability'] = 0
                
    total_individual_tokens = sum(value for key, value in individual_count.items())
    
    for tupl in gram_count:
        if gram == 1:
            gram_count[tupl]['probability'] = gram_count[tupl]['count'] / total_individual_tokens
        else:
            gram_count[tupl]['probability'] = gram_count[tupl]['count'] / individual_count[tupl[0]]
    
    return {'grams': gram_count, 'tokens': individual_count}

1.1. Construir el modelo probabilístico de unigramas y bigramas para:

```
corpus = [['A', 'B', 'C', 'D', 'E'],
          ['D', 'E', 'C', 'D', 'E'],
          ['A', 'C', 'D', 'D']
         ]
```
En este formato el corpus tiene tres frases y los términos o "tokens" están separados por comas (editar al formato que estén usando en su programa)

In [3]:
corpus = [
    ['A', 'B', 'C', 'D', 'E'],
    ['D', 'E', 'C', 'D', 'E'],
    ['A', 'C', 'D', 'D']
]

In [4]:
printer(corpus, grams(corpus, 1))

Corpus:
['A', 'B', 'C', 'D', 'E']
['D', 'E', 'C', 'D', 'E']
['A', 'C', 'D', 'D']

Tokens count:
A : 2
B : 1
C : 3
D : 5
E : 3

Grams
('A',) : {'count': 2, 'probability': 0.14285714285714285}
('B',) : {'count': 1, 'probability': 0.07142857142857142}
('C',) : {'count': 3, 'probability': 0.21428571428571427}
('D',) : {'count': 5, 'probability': 0.35714285714285715}
('E',) : {'count': 3, 'probability': 0.21428571428571427}


In [5]:
printer(corpus, grams(corpus, 2))

Corpus:
['A', 'B', 'C', 'D', 'E']
['D', 'E', 'C', 'D', 'E']
['A', 'C', 'D', 'D']

Tokens count:
A : 2
B : 1
C : 3
D : 5
E : 3

Grams
('A', 'B') : {'count': 1, 'probability': 0.5}
('B', 'C') : {'count': 1, 'probability': 1.0}
('C', 'D') : {'count': 3, 'probability': 1.0}
('D', 'E') : {'count': 3, 'probability': 0.6}
('E', 'C') : {'count': 1, 'probability': 0.3333333333333333}
('A', 'C') : {'count': 1, 'probability': 0.5}
('D', 'D') : {'count': 1, 'probability': 0.2}


In [6]:
printer(corpus, grams(corpus, 3))

Corpus:
['A', 'B', 'C', 'D', 'E']
['D', 'E', 'C', 'D', 'E']
['A', 'C', 'D', 'D']

Tokens count:
A : 2
B : 1
C : 3
D : 5
E : 3

Grams
('A', 'B', 'C') : {'count': 1, 'probability': 0.5}
('B', 'C', 'D') : {'count': 1, 'probability': 1.0}
('C', 'D', 'E') : {'count': 2, 'probability': 0.6666666666666666}
('D', 'E', 'C') : {'count': 1, 'probability': 0.2}
('E', 'C', 'D') : {'count': 1, 'probability': 0.3333333333333333}
('A', 'C', 'D') : {'count': 1, 'probability': 0.5}
('C', 'D', 'D') : {'count': 1, 'probability': 0.3333333333333333}


1.2. Construir un modelo probabilístico n-gram usando el programa, para dos documentos (pequeños) de su preferencia en el mismo idioma. 

In [7]:
# Texts taken from https://vitalik.ca/general/2019/04/03/collusion.html
text_A = '''However, that range of things that mechanisms of this type can do is limited. In the case of the content curation example above, we’re not really solving governance, we’re just scaling the functionality of a governance gadget that is already assumed to be trusted. One could try to replace the moderation panel with a prediction market on the price of a token representing the right to purchase advertising space, but in practice prices are too noisy an indicator to make this viable for anything but a very small number of very large decisions. And often the value that we’re trying to maximize is explicitly something other than maximum value of a coin.

Let’s take a more explicit look at why, in the more general case where we can’t easily determine the value of a governance decision via its impact on the price of a token, good mechanisms for identifying public goods and bads unfortunately cannot be identity-free or collusion-safe. If one tries to preserve the property of a game being identity-free, building a system where identities don’t matter and only coins do, there is an impossible tradeoff between either failing to incentivize legitimate public goods or over-subsidizing plutocracy.'''

text_B = '''The argument is as follows. Suppose that there is some author that is producing a public good (eg. a series of blog posts) that provides value to each member of a community of 10000 people. Suppose there exists some mechanism where members of the community can take an action that causes the author to receive a gain of $1. Unless the community members are extremely altruistic, for the mechanism to work the cost of taking this action must be much lower than $1, as otherwise the portion of the benefit captured by the member of the community supporting the author would be much smaller than the cost of supporting the author, and so the system collapses into a tragedy of the commons where no one supports the author. Hence, there must exist a way to cause the author to earn $1 at a cost much less than $1. But now suppose that there is also a fake community, which consists of 10000 fake sockpuppet accounts of the same wealthy attacker. This community takes all of the same actions as the real community, except instead of supporting the author, they support another fake account which is also a sockpuppet of the attacker. If it was possible for a member of the “real community” to give the author $1 at a personal cost of much less than $1, it’s possible for the attacker to give themselves $1 at a cost much less than $1 over and over again, and thereby drain the system’s funding. Any mechanism that can help genuinely under-coordinated parties coordinate will, without the right safeguards, also help already coordinated parties (such as many accounts controlled by the same person) over-coordinate, extracting money from the system.

A similar challenge arises when the goal is not funding, but rather determining what content should be most visible. What content do you think would get more dollar value supporting it: a legitimately high quality blog article benefiting thousands of people but benefiting each individual person relatively slightly, or this?'''

Tokenizacion

In [8]:
text_A = list(filter(lambda x: len(x) > 0, text_A.splitlines()))
text_B = list(filter(lambda x: len(x) > 0, text_B.splitlines()))

for idx, line in enumerate(text_A):
    text_A[idx] = WordPunctTokenizer().tokenize(line)
    
for idx, line in enumerate(text_B):
    text_B[idx] = WordPunctTokenizer().tokenize(line)

model_A = grams(text_A, 1)
model_B = grams(text_B, 1)

printer(text_A, model_A)
print('\n')
printer(text_B, model_B)

Corpus:
['However', ',', 'that', 'range', 'of', 'things', 'that', 'mechanisms', 'of', 'this', 'type', 'can', 'do', 'is', 'limited', '.', 'In', 'the', 'case', 'of', 'the', 'content', 'curation', 'example', 'above', ',', 'we', '’', 're', 'not', 'really', 'solving', 'governance', ',', 'we', '’', 're', 'just', 'scaling', 'the', 'functionality', 'of', 'a', 'governance', 'gadget', 'that', 'is', 'already', 'assumed', 'to', 'be', 'trusted', '.', 'One', 'could', 'try', 'to', 'replace', 'the', 'moderation', 'panel', 'with', 'a', 'prediction', 'market', 'on', 'the', 'price', 'of', 'a', 'token', 'representing', 'the', 'right', 'to', 'purchase', 'advertising', 'space', ',', 'but', 'in', 'practice', 'prices', 'are', 'too', 'noisy', 'an', 'indicator', 'to', 'make', 'this', 'viable', 'for', 'anything', 'but', 'a', 'very', 'small', 'number', 'of', 'very', 'large', 'decisions', '.', 'And', 'often', 'the', 'value', 'that', 'we', '’', 're', 'trying', 'to', 'maximize', 'is', 'explicitly', 'something', 'oth

 1.3  Comparar estadísticas de los dos "corpus". ¿Cuáles son las diferencias entre los unigramas más comunes de los dos conjuntos?
 
Se toman los n elementos más comunes para cada corpus. Se extrae del modelo el Token, el Contador, y su Probabilidad. Se puede notar que las palabras comunes con más frecuencia son: `the`, `of`, `a`, `to`, `,`, `.`, que resultan ser artículos y preposiciones, al igual que signos de puntuación. Se puede inferir que los tokens más comunes son de esta categoría, puesto que enlazan otros tokens con un significado de la idea a expresar.

In [9]:
sorted_A = sorted(model_A['tokens'].items(), key = lambda kv: (kv[1], kv[0]))
sorted_B = sorted(model_B['tokens'].items(), key = lambda kv: (kv[1], kv[0]))

items = 10
A = list(map(lambda x: list(x), sorted_A[-items:][::-1]))
B = list(map(lambda x: list(x), sorted_B[-items:][::-1]))

for idx, item in enumerate(A):
    A[idx].append(round(model_A['grams'][tuple([item[0]])]['probability'], 4))

for idx, item in enumerate(B):
    B[idx].append(round(model_B['grams'][tuple([item[0]])]['probability'], 4))

print('\t (Token, Count, Prob)\t<->\t (Token, Count, Prob)')
print('-------------------------------------------------------------')
for idx, item in enumerate(list(zip(A, B))):
    print(idx + 1, '\t', item[0], '\t<->\t', item[1])

	 (Token, Count, Prob)	<->	 (Token, Count, Prob)
-------------------------------------------------------------
1 	 ['the', 11, 0.0474] 	<->	 ['the', 29, 0.0753]
2 	 ['of', 10, 0.0431] 	<->	 ['of', 18, 0.0468]
3 	 ['a', 10, 0.0431] 	<->	 [',', 14, 0.0364]
4 	 [',', 8, 0.0345] 	<->	 ['a', 13, 0.0338]
5 	 ['to', 7, 0.0302] 	<->	 ['.', 11, 0.0286]
6 	 ['’', 6, 0.0259] 	<->	 ['community', 8, 0.0208]
7 	 ['.', 6, 0.0259] 	<->	 ['author', 8, 0.0208]
8 	 ['we', 4, 0.0172] 	<->	 ['1', 8, 0.0208]
9 	 ['that', 4, 0.0172] 	<->	 ['$', 8, 0.0208]
10 	 ['is', 4, 0.0172] 	<->	 ['to', 7, 0.0182]


In [10]:
list(zip([1, 2, 3], [4, 5, 6], [7, 8, 9]))

[(1, 4, 7), (2, 5, 8), (3, 6, 9)]